<center>
    <img src="https://upload.wikimedia.org/wikipedia/commons/6/6f/Dauphine_logo_2019_-_Bleu.png" style="width: 600px;"/> 
</center> 

<div align="center"><span style="font-family:Arial Black;font-size:33px;color:darkblue"> Master Economie Finance </span></div>

<div align="center"><span style="font-family:Arial Black;font-size:27px;color:darkblue">Application Lab – Risk measures </span></div>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

## Individual risk measures

### Value at risk(VaR)

#### Historical Simulation

The VaR at the $\alpha\%$ level is the $\alpha\%$ quantile of the return distribution $(F)$:

$$
VaR_{t}(\alpha) = F^{-1}_{r_{t}}(\alpha)
$$

In [ ]:
#Data import
df_ret_asset = pd.read_csv('ret_asset.csv')
df_ret_asset.info()

In [ ]:
df_ret_asset['Date'] = pd.to_datetime(df_ret_asset['Date'])
df_ret_asset.set_index('Date', inplace = True)
df_ret_asset

In [ ]:
alpha = 0.05
df_ret_asset.quantile(alpha)

#### Rolling window historical VaR

In [ ]:
def rolling_var_historical(series, alpha=0.05, window=250):
    return series.rolling(window).quantile(alpha)

rolling_var = rolling_var_historical(df_ret_asset, alpha=0.05, window=250)
rolling_var

#### Predicted VaR

$$ 
VaR^\alpha_{i, t+1 \mid t} = \sigma_{i, t+1 \mid t} \times q_{\alpha,t}
$$

where $\sigma_{i, t+1 \mid t}$ can be calculated based on Garch and $q_{\alpha,t} = F^{-1}_{\alpha} \left( { z_{i, t} } \right)$ is the conditional empirical $\alpha$-quantile of the standardized residuals $z_{i, t} = \frac{r_{i, t}}{\sigma_{i, t}}$.

In [ ]:
from arch import arch_model

In [ ]:
def compute_var(df_ret, window = 250, alpha = 0.05):

    # Create output DataFrames with same shape/index/columns
    df_Vol = 
    df_Q05 = 
    df_VaR = 

    for col in df_ret.columns:
        col_idx = df_ret.columns.get_loc(col) + 1
        for j in range(window, len(df_ret[col])):
            # Slice the rolling window for the current asset.
            rolling_data = 

            # Fit GARCH(1,1) with Zero mean and normal distribution
            garch_model = 

            # forecast(horizon=1).variance is σ^2，you MUST take the square root to get σ.
            sigma_forecast = 

            # In-window conditional volatility σ_t and standardized residuals z_t
            sigma_in = 
            z = 

            # Empirical alpha-quantile within window
            q_alpha = 

            # Save the results
            df_Vol.iloc[j, df_Vol.columns.get_loc(col)] = sigma_forecast
            df_Q05.iloc[j, df_Q05.columns.get_loc(col)] = q_alpha
            df_VaR.iloc[j, df_VaR.columns.get_loc(col)] = sigma_forecast * q_alpha

            # Print the row and column we are running

    return df_VaR, df_Vol, df_Q05

In [ ]:
df_VaR_asset, df_Vol_asset, df_Q05 = compute_var(df_ret_asset)

### VaR backtesting

In [ ]:
df_VaR_asset_bt = df_VaR_asset.iloc[250:]
df_VaR_asset_bt

In [ ]:
df_ret_asset_bt = df_ret_asset.iloc[250:]
df_ret_asset_bt

In [ ]:
from scipy.stats import chi2

### Unconditional coverage test

Kupiec (1995) introduced a variation on the binomial test called the proportion of failures (POF) test.  
The POF test works with the binomial distribution approach. In addition, it uses a likelihood ratio to test whether the probability of exceptions is synchronized with the probability \(p\) implied by the VaR confidence level.  
If the data suggests that the probability of exceptions is different than \(p\), the VaR model is rejected.  

The POF test statistic is

$$
LR_{POF} = -2 \log \left( \frac{(1-p)^{N-x} p^x}{\left(1-\frac{x}{N}\right)^{N-x} \left(\frac{x}{N}\right)^x} \right)
$$

where \(x\) is the number of failures, \(N\) the number of observations and \(p = VaR level\).  

This statistic is asymptotically distributed as a chi-square variable with 1 degree of freedom.  
The VaR model fails the test if this likelihood ratio exceeds a critical value.  
The critical value depends on the test confidence level.

### Independence test

Christoffersen (1998) proposed a test to measure whether the probability of observing an exception on a particular day depends on whether an exception occurred.  
Unlike the unconditional probability of observing an exception, Christoffersen's test measures the dependency between consecutive days only.  
The test statistic for independence in Christoffersen’s interval forecast (IF) approach is given by:

$$
LR_{CCI} = -2 \log \left( 
\frac{(1 - \pi)^{n00+n10} \pi^{n01+n11}}
{(1 - \pi_0)^{n00} \pi_0^{n01} (1 - \pi_1)^{n10} \pi_1^{n11}} 
\right)
$$

where

- \(n00\) = Number of periods with no failures followed by a period with no failures.  
- \(n10\) = Number of periods with failures followed by a period with no failures.  
- \(n01\) = Number of periods with no failures followed by a period with failures.  
- \(n11\) = Number of periods with failures followed by a period with failures.  

and

- $\pi_0$ — Probability of having a failure on period \(t\), given that no failure occurred on period \(t-1\):  
$$
  \pi_0 = \frac{n01}{n00 + n01}
$$

- $\pi_1$ — Probability of having a failure on period \(t\), given that a failure occurred on period \(t-1\):  
  $$
  \pi_1 = \frac{n11}{n10 + n11}
  $$

- $\pi$ — Probability of having a failure on period \(t\):  
  $$
  \pi = \frac{n01 + n11}{n00 + n01 + n10 + n11}
  $$


This statistic is asymptotically distributed as a chi-square variable with 1 degree of freedom.  


### CC test

Christoffersen (1998) also proposed the **Conditional Coverage (CC) test**, which jointly evaluates whether the frequency of exceptions is correct (Unconditional Coverage, UC) **and** whether exceptions occur independently (Independence, IND).  

The test statistic is simply the sum of the UC and IND likelihood ratio statistics:

$$
LR_{CC} = LR_{UC} + LR_{CCI}
$$


This statistic is asymptotically distributed as a chi-square variable with 2 degrees of freedom.

In [ ]:
def coverage_test_log_05(df_returns, df_VaR, alpha=0.05, test_alpha=0.05):
    results_list = []
    eps = 1e-12  # small epsilon to avoid log(0) / division by zero

    for col_name in df_VaR.columns:
        print(f"\nResults of {col_name}:")
        # Merge/align returns and VaR on the same date index.
        df_test = 
        # Extract the aligned return series and VaR series
        returns = 
        var_series = 

        #  Build the Exception indicator I_t = 1{ r_t < VaR_t } and drop NaN.
        I = 
        I = 
        T = 

        # ---------- UC (Kupiec) ----------
        # Count violations and non-violations
        n_1 = 
        n_0 = 
        pi_hat_uc = 

        # Guard against log(0) by clipping probabilities with eps.
        a0 = 
        a1 = 
        p0 = 
        p1 = 
        # UC likelihoods
        log_L_uc_H0 = 
        log_L_uc_H1 = 
        # UC LR stat and p-value
        LR_uc = 
        Pvalue_uc = 

        # ---------- Ind (Christoffersen independence) ----------
        # Build lag/lead of I_t to count transitions n_00, n_01, n_10, n_11
        I_lag = 
        I_lead = 
        n_00 = 
        n_01 = 
        n_10 = 
        n_11 = 
        
        # Denominators and totals for transition probabilities.
        denom0 = 
        denom1 = 
        total  = 

        # H0 overall failure prob and H1 transition probs
        pi_hat_ind = 
        pi_hat_01 = 
        pi_hat_11 =

        # Guard logs for Ind part
        q0 = 
        q1 = 
        r00 = 
        r01 = 
        r10 = 
        r11 = 
        # Ind likelihoods
        log_L_ind_H0 = 
        log_L_ind_H1 = 
        # Ind LR stat and p-value
        LR_ind = 
        Pvalue_ind = 

        # ---------- CC (Christoffersen conditional coverage) ----------
        
        LR_cc = 
        Pvalue_cc = 

        print(f"UC test:  LR = {LR_uc:.4f}, p-value = {Pvalue_uc:.4f}")
        print(f"Ind test: LR = {LR_ind:.4f}, p-value = {Pvalue_ind:.4f}")
        print(f"CC test:  LR = {LR_cc:.4f}, p-value = {Pvalue_cc:.4f}")

        results_list.append([col_name, Pvalue_uc, Pvalue_ind, Pvalue_cc])

    df_results = pd.DataFrame(results_list, columns=["Company", "UC p-value", "Ind p-value", "CC p-value"])

    df_summary = pd.DataFrame({
        f"p < {test_alpha}": [
            np.sum(df_results["UC p-value"]  < test_alpha),
            np.sum(df_results["Ind p-value"] < test_alpha),
            np.sum(df_results["CC p-value"]  < test_alpha)
        ],
        f"p ≥ {test_alpha}": [
            np.sum(df_results["UC p-value"]  >= test_alpha),
            np.sum(df_results["Ind p-value"] >= test_alpha),
            np.sum(df_results["CC p-value"]  >= test_alpha)
        ],
        "NaN": [
            np.sum(df_results["UC p-value"].isna()),
            np.sum(df_results["Ind p-value"].isna()),
            np.sum(df_results["CC p-value"].isna())
        ]
    }, index=["UC", "Ind", "CC"])

    df_reject = pd.DataFrame({
        f"Reject (<{test_alpha})": [
            df_results.loc[df_results["UC p-value"]  < test_alpha, "Company"].tolist(),
            df_results.loc[df_results["Ind p-value"] < test_alpha, "Company"].tolist(),
            df_results.loc[df_results["CC p-value"]  < test_alpha, "Company"].tolist()
        ]
    }, index=["UC", "Ind", "CC"])

    return df_results, df_summary, df_reject

In [ ]:
df_results_05, df_summary_05, df_reject_05 = coverage_test_log_05(df_ret_asset_bt, df_VaR_asset_bt, alpha=0.05, test_alpha = 0.05)
print(df_summary_05)
print(df_reject_05)

## CoVaR (Adrian and Brunnermeier, 2014)

The **CoVaR** is the Value-at-Risk of the market return given a specific event on the firm return:

$$
\Pr\left( r_{mt} \leq CoVaR_{t}^{m \mid r_{it} = VaR_{it}(\alpha)} \ \big|\ r_{it} = VaR_{it}(\alpha) \right) = \alpha.
$$


## ΔCoVaR

The contribution of the institution to systemic risk, **ΔCoVaR**, is the difference between its CoVaR and the CoVaR calculated at the median state:

$$
\Delta CoVaR_{it}(\alpha) = CoVaR_{t}^{m \mid r_{it} = VaR_{it}(\alpha)} - CoVaR_{t}^{m \mid r_{it} = Median(r_{it})}.
$$

---



Under the **Gaussian linear assumption**, the ΔCoVaR can be written as:

$$
\Delta CoVaR_{\alpha}^{m \mid i,t+1} 
= \rho_{m,i,t+1} \times \frac{\sigma_{m,t+1}}{\sigma_{i,t+1}} \times 
\Big( VaR_{\alpha}^{i,t+1} - \text{Median}(r_{i,t}) \Big).
$$

In [ ]:
# Market return
df_ret_mkt = pd.read_csv('ret_mkt.csv')
df_ret_mkt['Date'] = pd.to_datetime(df_ret_mkt['Date'])
df_ret_mkt.set_index('Date', inplace = True)
df_ret_mkt

In [ ]:
# Calculate the market predicted volatility using function compute_var()


# ρ in DCC(1,1): Engle (2002) Dynamic Conditional Correlation 

## 1) Covariance–correlation decomposition
Let $H_t$ be the $N\times N$ **conditional covariance** matrix of returns $\varepsilon_t$ (innovations):
$$
H_t = D_t\, R_t\, D_t,
$$
where $D_t=\mathrm{diag}\big(\sqrt{h_{1t}},\ldots,\sqrt{h_{Nt}}\big)$ collects **univariate conditional volatilities** (from any GARCH for each series), and $R_t$ is the **time-varying correlation matrix**.

---

## 2) Standardized residuals
Define standardized residuals
$$
u_t = D_t^{-1}\varepsilon_t = \big(u_{1t},\ldots,u_{Nt}\big)^\top, 
\qquad u_{it}=\frac{\varepsilon_{it}}{\sqrt{h_{it}}}.
$$

Let $\bar Q=\mathbb{E}[u_tu_t^\top]$ denote the **unconditional covariance** of $u_t$ (in practice, the sample covariance/correlation of standardized residuals) and assume $\bar Q$ is positive definite.

---

## 3) DCC(1,1) dynamics (Engle, 2002)
Introduce an auxiliary $N\times N$ symmetric positive definite matrix $Q_t$ that follows a GARCH-type recursion:
$$
\boxed{\quad
Q_t=(1-\alpha-\beta)\,\bar Q\;+\;\alpha\,u_{t-1}u_{t-1}^\top\;+\;\beta\,Q_{t-1},
\quad \alpha\ge 0,\;\beta\ge 0,\;\alpha+\beta<1.
\quad}
$$

The **dynamic correlation matrix** is then obtained by scaling $Q_t$ to unit diagonal:
$$
\boxed{\quad
R_t = \mathrm{diag}(Q_t)^{-1/2}\; Q_t\; \mathrm{diag}(Q_t)^{-1/2}.
\quad}
$$

---

## 4) Pairwise correlation coefficient $ \rho_{ij,t} $
Let $q_{ij,t}$ denote the $(i,j)$ element of $Q_t$.  
Then the DCC correlation between series $i$ and $j$ is
$$
\boxed{\quad
\rho_{ij,t}=\frac{q_{ij,t}}{\sqrt{q_{ii,t}\,q_{jj,t}}}.
\quad}
$$

---

## 5) Bivariate explicit formula 
For $N=2$, write $\bar Q=\begin{pmatrix}\bar q_{11}&\bar q_{12}\\ \bar q_{12}&\bar q_{22}\end{pmatrix}$.
The recursion for each element is
$$
\begin{aligned}
q_{12,t} &= (1-\alpha-\beta)\,\bar q_{12} + \alpha\,u_{1,t-1}u_{2,t-1} + \beta\,q_{12,t-1},\\[2pt]
q_{11,t} &= (1-\alpha-\beta)\,\bar q_{11} + \alpha\,u_{1,t-1}^2       + \beta\,q_{11,t-1},\\[2pt]
q_{22,t} &= (1-\alpha-\beta)\,\bar q_{22} + \alpha\,u_{2,t-1}^2       + \beta\,q_{22,t-1}.
\end{aligned}
$$
Hence the DCC(1,1) correlation is
$$
\boxed{\quad
\rho_{12,t}
=\frac{(1-\alpha-\beta)\,\bar q_{12}+\alpha\,u_{1,t-1}u_{2,t-1}+\beta\,q_{12,t-1}}
{\sqrt{\Big((1-\alpha-\beta)\,\bar q_{11}+\alpha\,u_{1,t-1}^2+\beta\,q_{11,t-1}\Big)
      \Big((1-\alpha-\beta)\,\bar q_{22}+\alpha\,u_{2,t-1}^2+\beta\,q_{22,t-1}\Big)}}.
\quad}
$$




The multivariate GARCH (MGARCH) in python is not well developped, so let's move to R just for illustration

In [ ]:
df_Rho = pd.read_csv('DCC_Rho.csv')
df_Rho['Date'] = pd.to_datetime(df_Rho['Date'])
df_Rho.set_index('Date', inplace = True)
df_Rho

In [ ]:
# Now calculate the Delta CoVaR as in the equation
